### 정렬, Aggregation 함수, Group By 적용
---
+ DataFrame, Series의 정렬 - sort_values()
  + RDMBS SQL의 order by 키워드와 매우 유사하다
  + sort_values()의 주요 입력 파라미터?
    + by, ascending, inplace
    + by: 정렬의 대상이 되는 칼럼
    + ascending: 오름차순 여부
    + inplace=False: 원본을 유지하며 정렬된 DataFrame 반환

In [2]:
import numpy as np
import pandas as pd

titanic_df = pd.read_csv('titanic_train.csv')

In [3]:
titanic_sorted = titanic_df.sort_values(by='Name')
titanic_sorted.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
845,846,0,3,"Abbing, Mr. Anthony",male,42.0,0,0,C.A. 5547,7.55,NaN,S
746,747,0,3,"Abbott, Mr. Rossmore Edward",male,16.0,1,1,C.A. 2673,20.25,NaN,S
279,280,1,3,"Abbott, Mrs. Stanton (Rosa Hunt)",female,35.0,1,1,C.A. 2673,20.25,NaN,S


In [4]:
titanic_sorted = titanic_df.sort_values(by=['Pclass', 'Name'], ascending=False)
titanic_sorted.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5,NaN,S
153,154,0,3,"van Billiard, Mr. Austin Blyler",male,40.5,0,2,A/5. 851,14.5,NaN,S
282,283,0,3,"de Pelsmaeker, Mr. Alfons",male,16.0,0,0,345778,9.5,NaN,S


#### Aggregation 함수 적용
---
+ min(), max(), sum(), count()와 같은 aggregation 함수의 적용은 RDBMS와 유사하다.
+ 다만 DataFrame의 경우, DataFrame에서 바로 호출할 경우 모든 칼럼에 해당 aggregation을 적용한다는 차이가 있다.
+ 아래 코드는 titanic_df에 count()를 적용한 결과를 보여준다

In [5]:
titanic_df.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [7]:
titanic_df[['Age', 'Fare']].mean()

Age     29.699118
Fare    32.204208
dtype: float64

#### Group by 적용
---
+ DataFrame의 groupby() 사용 시, 입력 파라미터 by에 컬럼을 입력하면 대상 컬럼으로 Groupby 된다.
+ 반환값은 DataFrameGroupBy라는 새로운 타입의 객체를 반환한다.

In [18]:
titanic_groupby = titanic_df.groupby(by='Pclass')
print(type(titanic_groupby))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


+ SQL의 groupby와 다르게, DataFrame에 groupby()를 호출해 반환된 결과에 aggregation 함수를 호출하면 groupby() 대상 칼럼을 제외한 모든 칼럼에 해당 aggregation 함수를 적용한다.
+ SQL의 경우 group by 적용 시 여러 개의 컬럼에 aggregation 함수를 호출하려면 대상 컬럼을 모두 Select 절에 나열해야 한다
  - Select count(PassengerId), count(Survived), ... from titanic_table group by Pclass
+ DataFrame에서는, 특정 컬럼만 이용해서 groupby() 호출한 뒤, 반환된 DataFrameGroupBy 객체에 aggregation 함수를 적용한다.

In [21]:
titanic_groupby = titanic_df.groupby('Pclass').count()
print(type(titanic_groupby))
titanic_groupby

<class 'pandas.core.frame.DataFrame'>


,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Pclass,,,,,,,,,,,
1,216,216,216,216,186,216,216,216,216,176,214
2,184,184,184,184,173,184,184,184,184,16,184
3,491,491,491,491,355,491,491,491,491,12,491


In [22]:
titanic_groupby = titanic_df.groupby('Pclass')[['PassengerId', 'Survived']].count()
titanic_groupby

,PassengerId,Survived
Pclass,,
1,216,216
2,184,184
3,491,491


+ 여러 aggregation 함수를 적용하기 위해서는?
  - SQL이라면, Select 절에 나열
    - Select max(Age), min(Age) from titanic_table
  - DataFrame에서는 적용하고자 하는 aggregation 함수명들을 DataFrameGroupBy 객체의 agg() 함수에 인자로 넣어준다

In [24]:
titanic_df.groupby('Pclass')['Age'].agg([max, min])

,max,min
Pclass,,
1,80.0,0.92
2,70.0,0.67
3,74.0,0.42


+ 다른 컬럼에 각각 다른 aggregation 함수를 적용하기에는 힘들다는 단점이 있다.
+ 이를 해결하기 위해, agg 내에 입력값을 딕셔너리 형태로 컬럼: 함수명의 형태로 넣어준다

In [25]:
agg_format = {'Age': 'max', 'SibSp': 'sum', 'Fare':'mean'}
titanic_df.groupby('Pclass').agg(agg_format)

,Age,SibSp,Fare
Pclass,,,
1,80.0,90,84.154687
2,70.0,74,20.662183
3,74.0,302,13.675550


#### 결손 데이터 처리하기
---
+ pandas는 결손 데이터(Missing Data)를 처리하는 API를 제공한다. 
+ 결손 데이터는 칼럼에 값이 없는, 즉 NULL인 경우를 의미한다.
+ 이는 Numpy의 NaN으로 표시하며, 머신러닝 알고리즘은 이 값을 처리하지 않으므로 다른 값으로 대체해야 한다.
+ NaN 값은 평균, 총합 등의 함수 연산 시 제외가 된다.
+ 특정 컬럼의 100개 데이터 중 10개가 NaN 값일 경우, 이 칼럼의 평균 값은 90개 데이터에 대한 평균이다.
+ 이를 확인하는 API는 **isna**()이며, NaN값을 다른 값으로 대체하는 API는 **fillna()**이다.

In [26]:
titanic_df.isna().head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False


+ sum()을 호출 시, True는 내부적으로 1, False는 0으로 변환되어 결손 데이터의 갯수를 구할 수 있다.

In [27]:
titanic_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

+ fillna()를 이용해 결손 데이터를 편리하게 다른 값으로 대체할 수 있다.
+ 아래의 코드는 Cabin 칼럼의 Nan 값을 'C000'으로 대체한다.

In [29]:
titanic_df['Cabin'] = titanic_df['Cabin'].fillna('C000')
titanic_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,C000,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,C000,S


+ fillna()를 이용해 반환 값을 다시 받거나, inplace=True 파라미터를 fillna()에 추가해야 실제 데이터 세트의 값이 변경된다.
+ 아래는 'Age' 컬럼의 NaN 값을 평균 나이로, 'Embarked' 컬럼의 NaN 값을 S로 대체해 모든 결손데이터를 처리하는 코드

In [30]:
titanic_df['Age'] = titanic_df['Age'].fillna(titanic_df['Age'].mean())
titanic_df['Embarked'] = titanic_df['Embarked'].fillna('S')
titanic_df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

#### apply lambda 식을 활용한 데이터 가공
---
+ pandas는 apply 함수에 람다 식을 결합해 DataFrame이나 Series의 레코드 별로 데이터를 가공하는 기능을 제공한다.
+ 컬럼에 일괄적으로 데이터 가공을 하는 것이 속도면에서 빠르지만, 복잡한 데이터 가공이 필요할 경우 이 기능을 사용한다.
+ 람다(lambda) 식이란?
  - 파이썬에서 함수형 프로그래밍을 지원하기 위해서 만들어짐
  - 다음과 같이 입력값의 제곱을 구해서 반환하는 get_square(a) 함수가 있다고 가정해보자.
  
    ```python
    def get_square(a):
          return a**2
    print('3의 제곱은:', get_square(3))
    # 3의 제곱은: 9
    ```
  
  - 위의 함수는 def get_square(a):와 같이 함수 명과 입력 인자를 먼저 선언하고 함수 내에서 입력 인자를 가공한 뒤 결과값으로 반환한다.
  - lambda는 이러한 함수의 선언과 함수 내의 처리를 한 줄의 식으로 쉽게 변환하는 식
  
    ```python
    lambda_square = lambda x : x ** 2
    print('3의 제곱은: ', lambda_square(3))
    # 3의 제곱은: 9
    ```
  
  - lambda x : x ** 2에서 ':'로 입력 인자와 반환될 입력 인자의 계산식을 분리한다.
  - 왼쪽의 x는 입력 인자를 가리키며, 오른쪽은 입력 인자의 계산식이다. 결국 오른쪽은 반환 값을 의미하는 것
  - 여러 개의 값을 입력 인자로 사용해야 할 경우에는 보통 map() 함수를 결합한다.
  
    ```python
    a = [1, 2, 3]
    squares = map(lambda x : x ** 2, a)
    list(squares)
    # [1, 4, 9]
    ```

+ pandas의 람다 식은 파이썬의 람다 식을 그대로 적용한 것.
+ Name 컬럼의 문자열 개수(길이)를 별도의 컬럼인 Name_len에 저장해보자

In [31]:
titanic_df['Name_len'] = titanic_df['Name'].apply(lambda x : len(x))
titanic_df[['Name', 'Name_len']].head(3)

,Name,Name_len
0,"Braund, Mr. Owen Harris",23
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",51
2,"Heikkinen, Miss. Laina",22


+ 나이가 15세 미만이면 Child, 그렇지 않으면 Adult로 구분하는 새로운 컬럼 Child_Adult를 만드는 코드
+ 람다 식에서는 if else를 지원하는데, if절의 경우 if식보다 반환 값을 먼저 기술해야 한다. 이는 :의 오른편에 반환값이 바로 있어야 하기 때문
+ else if는 지원하지 않는다.

In [34]:
titanic_df['Child_Adult'] = titanic_df['Age'].apply(lambda x : 'Child' if x < 15 else 'Adult')
titanic_df[['Age', 'Child_Adult']].head(8)

,Age,Child_Adult
0,22.000000,Adult
1,38.000000,Adult
2,26.000000,Adult
3,35.000000,Adult
4,35.000000,Adult
5,29.699118,Adult
6,54.000000,Adult
7,2.000000,Child


+ 나이가 15세 이하이면 Child, 15세~60세 사이는 Adult, 61세 이상은 Elderly로 분류하는 컬럼 만들기

In [35]:
titanic_df['Age_cat'] = titanic_df['Age'].apply(lambda x : 'Child' if x < 15 else ('Adult' if x <= 60 else 'Elderly'))
titanic_df['Age_cat'].value_counts()

Adult      791
Child       78
Elderly     22
Name: Age_cat, dtype: int64

+ 조건이 많아지면 부담스러울 수 있기 때문에, 별도의 함수를 만드는 게 더 나을 수 있다.

In [36]:
def get_category(age):
    cat = ''
    if age <= 5: cat = 'Baby'
    elif age <= 12: cat = 'Child'
    elif age <= 18: cat = 'Teenager'
    elif age <= 25: cat = 'Student'
    elif age <= 35: cat = 'Young Adult'
    elif age <= 60: cat = 'Adult'
    else: cat = 'Elderly'
    return cat

titanic_df['Age_cat'] = titanic_df['Age'].apply(lambda x : get_category(x))
titanic_df[['Age', 'Age_cat']].head()

,Age,Age_cat
0,22.0,Student
1,38.0,Adult
2,26.0,Young Adult
3,35.0,Young Adult
4,35.0,Young Adult
